In [29]:
import os
import sys
import itertools
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd
from scipy.stats import spearmanr

from matplotlib import pyplot as plt
import matplotlib_venn
import seaborn as sns

import torch
from torch import nn, optim
from transformers import BertTokenizer, BertModel, BertForMaskedLM

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# re-load functions
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [3]:
DEVICE = 'cpu'

## convert to augmented trn

In [4]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [6]:
trn_df = pd.read_csv('../mnt/inputs/origin/train.csv')
display(trn_df.shape, trn_df.head())
tst_df = pd.read_csv('../mnt/inputs/origin/test.csv')
display(tst_df.shape, tst_df.head())
sub_df = pd.read_csv('../mnt/inputs/origin/sample_submission.csv')
display(sub_df.shape, sub_df.head())

(6079, 41)

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.000000,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.000000,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com,0.888889,0.666667,0.666667,1.0,1.0,1.0,0.444444,0.444444,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com,1.000000,0.666667,0.000000,1.0,1.0,1.0,0.666667,0.666667,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,1.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


(476, 11)

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


(476, 31)

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [12]:
aug_rows = []

aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")
aug2 = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

for i, row in tqdm(list(trn_df.iterrows())):
    try:
        _row = row.copy()
        _row['question_title'] = aug.augment(_row['question_title'])
        _row['question_body'] = aug.augment(_row['question_body'])
        _row['answer'] = aug.augment(_row['answer'])
        aug_rows.append(_row)
        _row2 = row.copy()
        _row2['question_title'] = aug2.augment(_row2['question_title'])
        _row2['question_body'] = aug2.augment(_row2['question_body'])
        _row2['answer'] = aug.augment(_row2['answer'])
        aug_rows.append(_row2)
    except Exception as e:
        print(e)
    # break

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for +=: 'NoneType' and 'str'
unsupported operand type(s) for

In [9]:
aug2.augment(_row2['question_body'])

TypeError: unsupported operand type(s) for +=: 'NoneType' and 'str'

In [14]:
_row.question_title

'what am we losing by using extension tubes instead of a rigid structure ?'

In [15]:
_row2.question_title

'what am i losing when into extension tubes , without the macro lens ?'

In [13]:
res_df = pd.concat(aug_rows, axis=0)
res_df.shape

(497699,)

In [20]:
import pickle
with open('../mnt/inputs/aug_rows.pkl', 'wb') as fout:
    pickle.dump(aug_rows, fout)

In [23]:
res_df = pd.DataFrame(aug_rows)
res_df.shape

(12139, 41)

In [25]:
res_df.to_pickle('../mnt/inputs/nes_info/aug_train_1.pkl')

## id 調整 for grouping

In [4]:
aug_df = pd.read_pickle('../mnt/inputs/nes_info/aug_train_1.pkl')
display(aug_df.shape, aug_df.head())

(12139, 41)

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,how was i losing ? use extension tubes instead...,after playing around with macro - on - the - c...,ysap,https://photo.stackexchange.com/users/1024,"i just got extension tubes , so here ' s the s...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
0,0,and what more am i losing when using extension...,after playing around with macro photography an...,ysap,https://photo.stackexchange.com/users/1024,"i just got extension tubes , so here ' s the s...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,what is the difference between the city versus...,i was trying to understand what kind of places...,russellpierce,https://rpg.stackexchange.com/users/8774,it might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.0,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
1,1,"what is not the common distinction , between b...",i perhaps am also trying to understand what ki...,russellpierce,https://rpg.stackexchange.com/users/8774,here might be helpful a look into the definiti...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.0,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,maximum protusion length ; post - hole compone...,i ' m equipped with a pcb that has through - h...,Joe Baker,https://electronics.stackexchange.com/users/10157,do you even need grooves ? we make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.0,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889


In [7]:
trn_df.question_body.value_counts()

Which is the best introductory textbook for Bayesian statistics?\n\nOne book per answer, please.\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [8]:
aug_df.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,question_opinion_seeking,question_type_choice,question_type_compare,question_type_consequence,question_type_definition,question_type_entity,question_type_instructions,question_type_procedure,question_type_reason_explanation,question_type_spelling,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,how was i losing ? use extension tubes instead...,after playing around with macro - on - the - c...,ysap,https://photo.stackexchange.com/users/1024,"i just got extension tubes , so here ' s the s...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
0,0,and what more am i losing when using extension...,after playing around with macro photography an...,ysap,https://photo.stackexchange.com/users/1024,"i just got extension tubes , so here ' s the s...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com,1.000000,0.333333,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,what is the difference between the city versus...,i was trying to understand what kind of places...,russellpierce,https://rpg.stackexchange.com/users/8774,it might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.0,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
1,1,"what is not the common distinction , between b...",i perhaps am also trying to understand what ki...,russellpierce,https://rpg.stackexchange.com/users/8774,here might be helpful a look into the definiti...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com,1.000000,1.000000,0.0,0.5,1.0,1.0,0.444444,0.444444,0.666667,0.0,0.000000,0.666667,0.666667,0.0,0.333333,0.0,0.0,0.000000,0.333333,0.0,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,maximum protusion length ; post - hole compone...,i ' m equipped with a pcb that has through - h...,Joe Baker,https://electronics.stackexchange.com/users/10157,do you even need grooves ? we make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com,0.888889,0.666667,0.0,1.0,1.0,1.0,0.666667,0.444444,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,1.0,0.333333,0.333333,0.0,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889


In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [11]:
trn_df['question_body_le'] = le.fit_transform(trn_df.question_body)

In [12]:
trn_df['question_body_le'].value_counts()

3442    12
1525    11
3358    11
3431     9
1503     7
        ..
2347     1
298      1
2335     1
286      1
2045     1
Name: question_body_le, Length: 3583, dtype: int64

In [22]:
trn_df.to_pickle('../mnt/inputs/nes_info/trn_df.pkl')

In [23]:
from functools import partial
from multiprocessing import Pool

In [30]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [25]:
def txt_augment(row_pair, aug):
    i, row = row_pair
    row = row.copy()
    
    try:
        row['question_title'] = aug.augment(row['question_title'])
        row['question_body'] = aug.augment(row['question_body'])
        row['answer'] = aug.augment(row['answer'])    
        return row

    except Exception as e:
        print(e)

In [27]:
trn_df = pd.read_pickle('../mnt/inputs/nes_info/trn_df.pkl')

In [31]:
with Pool(os.cpu_count()) as p:
    aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")

    iter_func = partial(
        txt_augment,
        aug=aug,
        )
    imap = p.imap_unordered(iter_func, trn_df.iterrows())
    res = list(tqdm(imap, total=len(trn_df)))
    p.close()
    p.join()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [39]:
ContextualWordEmbsAug_sub_df = pd.DataFrame(res).reset_index(drop=True)# pd.concat(res, axis=0).reset_index(drop=True)
ContextualWordEmbsAug_sub_df.to_pickle('../mnt/inputs/nes_info/ContextualWordEmbsAug_sub_df.pkl')

In [33]:
trn_df.question_body_le.isnull().mean()

0.0

In [33]:
trn_df.question_body_le.isnull().mean()

0.0

In [40]:
ContextualWordEmbsAug_sub_df.shape

(6079, 42)

In [35]:
res[0]

qa_id                                                                                    0
question_title                           what was i losing when using extension functio...
question_body                            after playing around with macro photography on...
question_user_name                                                                    ysap
question_user_page                              https://photo.stackexchange.com/users/1024
answer                                   i just got extension tubes , so here ' s the s...
answer_user_name                                                                    rfusca
answer_user_page                                https://photo.stackexchange.com/users/1917
url                                      http://photo.stackexchange.com/questions/9169/...
category                                                                         LIFE_ARTS
host                                                               photo.stackexchange.com

In [ ]:
with Pool(os.cpu_count()) as p:
    aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="insert")

    iter_func = partial(
        txt_augment,
        aug=aug,
        )
    imap = p.imap_unordered(iter_func, trn_df.iterrows())
    res = list(tqdm(imap, total=len(trn_df)))
    p.close()
    p.join()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [ ]:
ContextualWordEmbsAug_ins_df = pd.concat(res, axis=0).reset_index(drop=True)
ContextualWordEmbsAug_ins_df.to_pickle('../mnt/inputs/nes_info/ContextualWordEmbsAug_ins_df.pkl')